In [ ]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar, get_defaults, reset_cache, Animation
from jupyter_cadquery.cadquery import show, Assembly, Part, Faces, Edges
from jupyter_cadquery.mate_assembly import Mate, MAssembly

set_sidecar("Hexapod")
set_defaults(axes=True, axes0=True, edge_accuracy=0.01, mate_scale=5, zoom=3)

# Hexapod 

![2-hexapod.png](2-hexapod.png)

In [ ]:
import numpy as np
horizontal_angle = 25

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1)
    
def vertical(count, end, offset):
    ints = intervals(count)
    heights = [round(45 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = -1 if reverse else 1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

print("Leg group 1 (transparent)")
print("horizontal movement", horizontal(4, True))
print("vertical heights   ", vertical(8, 4, 0))

print("\nLeg group 1 (filled)")
print("horizontal movement", horizontal(4, False))
print("vertical heights   ", vertical(8, 4, 4))

# Assembly

## Parts

In [ ]:
thickness = 2
height = 40
width = 65
length = 100
diam = 4
tol = 0.05

In [ ]:
x1, x2 = 0.63, 0.87
base_holes = {
    "right_back": (-x1*length, -x1*width), "right_middle": (0, -x2*width), "right_front": ( x1*length, -x1*width),
    "left_back":  (-x1*length,  x1*width), "left_middle":  (0,  x2*width), "left_front":  ( x1*length,  x1*width),
}
stand_dist = {"front_stand": 0.75 * length, "back_stand": -0.8 * length}

base = (cq.Workplane()
    .ellipse(length, width).pushPoints(list(base_holes.values())).circle(diam / 2 + tol)
    .extrude(thickness)
)

stand_cutout = cq.Workplane().box(thickness + 2 * tol, width / 2 + 2 * tol, thickness * 2)
front_cutout = cq.Workplane().box(length / 3, 2 * (width + tol), 2 * thickness)

base = (base
    .cut(front_cutout.translate((length, 0, 0)))
    .cut(stand_cutout.translate((stand_dist["back_stand"], 0, thickness)))
    .cut(stand_cutout.translate((stand_dist["front_stand"], 0, thickness)))
    .faces(">X").edges("not |Y").fillet(width / 5)
)

In [ ]:
stand = cq.Workplane().box(height, width / 2 + 10, thickness)
inset = cq.Workplane().box(thickness, width / 2, thickness)
backing = cq.Workplane("ZX").polyline([(10,0), (0,0), (0, 10)]).close().extrude(thickness)

stand = (stand
    .union(inset.translate(( (height + thickness) / 2, 0, 0)))
    .union(inset.translate((-(height + thickness) / 2, 0, 0)))
    .union(backing.translate((-height / 2, -thickness / 2, thickness / 2)))
    .union(backing.rotate((0, 0, 0), (0, 1, 0), -90).translate((height / 2, -thickness / 2, thickness / 2)))
)


In [ ]:
l1, l2 = 50, 80
pts = [( 0,  0), ( 0, height/2), (l1, height/2 - 5), (l2, 0)]
upper_leg_hole = (l2 - 10, 0)

upper_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([upper_leg_hole]).circle(diam/2 + tol).extrude(thickness)
    .edges("|Z and (not <X)").fillet(4)
)

axle = (cq.Workplane("XZ", origin=(0, height/2 + thickness + tol, thickness/2))
    .circle(diam/2).extrude(2 * (height/2 + thickness + tol))
)

upper_leg = upper_leg.union(axle)

In [ ]:
w, l1, l2 = 15, 20, 120
pts = [( 0,  0), ( l1, w), (l2, 0)]
lower_leg_hole = (l1 - 10, 0)

lower_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([lower_leg_hole]).circle(diam/2 + tol)
    .extrude(thickness)
    .edges("|Z").fillet(5)
)

In [ ]:
leg_angles = {
    "right_back": 105,  "right_middle": 90, "right_front": 75, 
    "left_back": -105,  "left_middle": -90, "left_front": -75,
}
leg_names = list(leg_angles.keys())
stand_names = ("front_stand", "back_stand")

In [ ]:
show(
    base, 
    stand.translate((0,100,thickness/2)), 
    upper_leg.translate((-100,-100,0)),
    lower_leg.translate((0,-100,0)),
)

## Define Assembly

In [ ]:
def create_hexapod():
    # Some shortcuts
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda *args: cq.Color(*args)

    # Leg assembly
    leg = (MAssembly(upper_leg, name="upper", color=C("orange"))
        .add(lower_leg, name="lower", color=C("orange"), loc=L(80,0,0))
    )
    # Hexapod assembly
    hexapod = (MAssembly(base, name="bottom", color=C("gray"), loc=L(0, 1.1*width, 0))
        .add(base, name="top", color=C(0.9, 0.9, 0.9), loc=L(0, -2.2*width, 0))
        .add(stand, name="front_stand", color=C(0.5, 0.8, 0.9), loc=L( 40, 100, 0))
        .add(stand, name="back_stand", color=C(0.5, 0.8, 0.9), loc=L(-40, 100, 0))
    )
    for i, name in enumerate(leg_names):
        hexapod.add(leg, name=name, loc=L(100, -55*(i-1.7), 0))

    return hexapod

## Define Mates 

In [ ]:
hexapod = create_hexapod()

M = lambda selector, *obj_selectors: Mate(hexapod.find(selector, *obj_selectors))

hexapod.mate(
    name="bottom", selector="bottom", origin=True,
    mate=M("bottom", "faces@>Z")
).mate(
    name="top", selector="top", origin=True, 
    mate=M("top", "faces@<Z").rx(180).tz(-(height + 2 * tol))
)

angle = {"front_stand":180, "back_stand":0}
for name in stand_names:
    hexapod.mate(
        name=f"bottom_{name}", selector="bottom", 
        mate=M("bottom", "faces@<Z", ("wires", (stand_dist[name], 0))).rz(angle[name]).rx(180)
    ).mate(
        name=f"{name}_lower", selector=name, origin=True,
        mate=M(name, "faces@<X").rz(90)
    )

for name, pnt in base_holes.items():
    hexapod.mate(
        name=f"bottom_{name}", selector="bottom", 
        mate=M("bottom", "faces@<Z", ("wires", pnt)).rx(180).rz(leg_angles[name])
    )

for name in leg_names:
    lower, upper = (">Z", "<Z") if "left" in name else ("<Z", ">Z")
    hexapod.mate(
        name=f"{name}_lower_hinge",  selector=f"{name}>lower", origin=True, 
        mate=M(f"{name}>lower", ("faces", lower), ("wires", lower_leg_hole))
    ).mate(
        name=f"{name}_upper_hinge",  selector=name,
        mate=M(name, ("faces", upper), ("wires", upper_leg_hole)).rz(-75)
    ).mate(
        name=f"{name}_bottom_hinge", selector=name, origin=True,
        mate=M(name, "faces@<Y")
    )

show(hexapod, render_mates=True)

## Relocate and assemble

In [ ]:
hexapod.relocate()
show(hexapod, render_mates=True)

In [ ]:
for leg in leg_names:
    hexapod.assemble(f"{leg}_bottom_hinge", f"bottom_{leg}")
    hexapod.assemble(f"{leg}_lower_hinge", f"{leg}_upper_hinge")

for stand_name in stand_names:
    hexapod.assemble(f"{stand_name}_lower", f"bottom_{stand_name}")

hexapod.assemble("top", "bottom")

d = show(hexapod, render_mates=False, grid=True, axes=False)

## Animation

In [ ]:
leg_group = ("left_front", "right_middle", "left_back")

animation = Animation(d.root_group)

for name in leg_names:
    # move upper leg
    animation.add_number_track(f"bottom>{name}", "rz", *horizontal(4, "middle" in name))
    
    # move lower leg
    animation.add_number_track(f"bottom>{name}>lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4))
    
    # lift hexapod to run on grid
    animation.add_number_track(f"bottom", "tz", [0, 4], [61.25]*2)
    
animation.animate(speed=5)